# Chapter 1, Derivatives

The book **Deep Learning from Scratch** is a great introduction to Deep Learning.  It uses Python to build a deep learning toolkit from scratch (as the name implies).

To better learn the material though, I'm translating it to F#.  This will force me to learn the material "from scratch", so I can't cheat and just run the Python notebooks.  Though I might do that to compare outputs.

First, we need to load our libraries.  

For the `pandas` substitute, I'm using `Deedle`.  For `matplotlib`, I'm using `XPlot.Plotly` (and `XPlot.Plotly.Interactive` for notebook integration).  

In [1]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Deedle"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Deedle, 2.5.0 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

## Math

The book begins with, and we will too, math.

Calculus and linear algebra are at the heart of Deep Learning.  And calculus starts with derivatives. (So cliché, which is of itself derivative. ☺)

The first chapter of the book covers calculus: single parameter function derivatives through vector calculus.  But the introduction is gentle and done with code along side math notation and some interesting box diagrams.

We're going to stick with code.

### Derivatives

Up first in any calculus discussion are derivatives.  In short, they are the slope of the line tangent to point $x$ for $f$.  There are much better explanations. One of my favorites is [Calculus Made Easy](https://www.calculusmadeeasy.org/) by Silvanus P. Thompson.

If we make the $\delta$ (delta) very small, we can get a line segment for the points around $x$ and take its slope, approximating to an arbitrarily accurate degree the derivative of $f(x)$.

In [2]:
open Deedle
open System.Linq

In [3]:
let deriv delta f x = 
    ((f (x + delta)) - (f (x - delta))) / (2.0 * delta)

// partial application for a default delta
let dydx = deriv 0.001


Our `deriv` function simply takes a the slope of a line segment of $f$ at the $x$.  The `dydx` function is simply a partial application with the delta of 0.001.  Which is why I put the `delta` parameter first.

Let's try out some simple charts to make sure we get what we think we should for derivatives.  Let's plot the functions $x^2$ and $x^3$ and their derivatives.

In [4]:
open XPlot.Plotly
open XPlot.Plotly.Interactive

let x = seq { for n in -3.0 .. 0.1 .. 3.0 do n }
let x2 = seq { for n in -3.0 .. 0.1 .. 3.0 do n ** 2.0 }
let x3 = seq { for n in -3.0 .. 0.1 .. 3.0 do n ** 3.0 }
let dx2 = seq { for n in -3.0 .. 0.1 .. 3.0 do dydx (fun x -> (x ** 2.0)) n }
let dx3 = seq { for n in -3.0 .. 0.1 .. 3.0 do dydx (fun x -> (x ** 3.0)) n }

In [5]:
let line1 = Scatter(
        x = x, 
        y = x2,
        mode = "lines",
        name = "x^2"
    )

let dline1 = Scatter(
        x = x, 
        y = dx2,
        mode = "lines",
        name = "dy/dx x^2"
    )

let lineLayout = Layout(title = "Derivative of x**2 Test Chart")

let chart = 
    [line1; dline1]
    |> Chart.Plot
    |> Chart.WithLayout lineLayout
    |> Chart.WithWidth 750

chart

<!DOCTYPE html>

In [9]:
let line2 = Scatter(
        x = x, 
        y = x3,
        mode = "lines",
        name = "x^3"
    )


let dline2 = Scatter(
        x = x, 
        y = dx3,
        mode = "lines",
        name = "dy/dx x^3"
    )

let lineLayout = Layout(title = "Derivative of x**3 Test Chart")

let chart = 
    [line2; dline2]
    |> Chart.Plot
    |> Chart.WithLayout lineLayout
    |> Chart.WithWidth 750

chart

<!DOCTYPE html>

Seems pretty good.  

For $x^2$ we see the familiar, if flat, parabola, and the derivative predictably starts negative, since the parabola `y` is decreasing. Since the slope is becoming flatter, the derivative becomes closer to 0, until at 0, where the parabola begins its ascent, likewise the derivative begins to become positive.  These changes in slope manifest as a simple line crossing the axes at 0.

For $x^3$ we see the cubic parabola (I had to look that up).  This is interesting because our slop at -3 is positive, so the derivative is positive.  The slope of the cubic parabola is decreasing though, so the derivative likewise is decreasing.  Again, at 0, the cubic parabola becomes flat, so the derivative is 0.  And like the parabola, the cubic parabola ascends, so the derivative tracks that ascent. These changes manifest as a parabola with the change in slope from negative to positive at 0.

Creating this function in F# required extra parentheses that the Python function in Deep Learning from Scratch did not.  Not sure why.  I would have that the order of operations for `/ 2.0 * delta` would be the same as `/ (2.0 * delta)`, but maybe there is something there in floating point precision or the operators in F# that I don't understand yet.

From my college calculus course, I remember that the derivatives of the parabola and cubic parabola follow the power rule, which says that $\frac{dy}{dx} x^n = n x^{n-1}$.  Let's plot those and verify that our derivatives looks similar.

In [16]:
let power_rule_x2 = x |> Seq.map (fun y -> y * 2.0)

let dydx2 = Scatter(
        x = x, 
        y = power_rule_x2,
        mode = "lines",
        name = "dy/dx x^2"
    )

let power_rule_x3 = x |> Seq.map (fun y -> 3.0 * (y ** 2.0))

let dydx3 = Scatter(
        x = x, 
        y = power_rule_x3,
        mode = "lines",
        name = "dy/dx x^3"
    )

let lineLayout = Layout(title = "Power Rule Derivative Test Chart")

let chart = 
    [dydx2; dydx3]
    |> Chart.Plot
    |> Chart.WithLayout lineLayout
    |> Chart.WithWidth 750

chart

<!DOCTYPE html>

Based on my recollection of the power rule from college, these do indeed look correct.

I am going to call our derivative function a success.

As a bonus, we can see from the power rule, that indeed the derivative of $x^2$ should be a line, 1 degree less than the square.  Likewise, for $x^3$, we see that the derivative should be a parabola.  So it matches in form and function.

In [15]:
open Deedle;

let d = Series.ofValues (seq { for n in -3.0 .. 0.1 .. 3.0 do n })
let sc = Scatter(
        x = d.Keys, 
        y = d.Values,
        mode = "lines",
        name = "Deedle Test"
    )

let lineLayout = Layout(title = "Blah blah blah")

let ch = [sc] |> Chart.Plot
ch

<!DOCTYPE html>